Notebook for plotting Corona data from SSI togeter with temperature data from DMI
This was just for fun, suggested by a frind (Franz hence the name).,

Run SSI_get_data first to get a dataset of the right format.
SSI Data are read from subfolder : data
Date of current SSI dataset is read from : data_date.dat 

Temperature data can be downloaded from https://www.dmi.dk/vejrarkiv/ (you need to do a bit of klicking to get mean temperature on a month basis.

This notebook is not in any way robust, or generic, some stuff is just handcoded to the case of the actual data. 


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from pathlib import Path
from datetime import date, timedelta
import pickle
from matplotlib import cm

In [ ]:
# Read date of last data-download SSI data
f = open("data_date.dat", 'rb')
date_str=pickle.load(f)
f.close()

print("Dataset is from: " + date_str)

#define the file to read and read it
datafolder=Path("data/")
datafile=datafolder / "Test_pos_over_time.csv"
df=pd.read_csv(datafile,  sep=';', parse_dates=['Date'], index_col=['Date'],error_bad_lines=False, engine='python', skipfooter=2, decimal=',', thousands='.')

In [ ]:
# Read temperature data 

datafile="hele-landet-december-2020.csv"
df_temp=pd.read_csv(datafile, parse_dates=['DateTime'], index_col=['DateTime'],quotechar='"', sep=';',error_bad_lines=False, engine='python', decimal=',', thousands='.', encoding = 'utf8')

datafile="hele-landet-november-2020.csv"
x=pd.read_csv(datafile, parse_dates=['DateTime'], index_col=['DateTime'],quotechar='"', sep=';',error_bad_lines=False, engine='python', decimal=',', thousands='.', encoding = 'utf8')
df_temp=df_temp.append(x)

datafile="hele-landet-januar-2021.csv"
x=pd.read_csv(datafile, parse_dates=['DateTime'], index_col=['DateTime'],quotechar='"', sep=';',error_bad_lines=False, engine='python', decimal=',', thousands='.', encoding = 'utf8')
df_temp=df_temp.append(x)



In [ ]:
# merge the two datasets based on date index
merge=pd.merge(df,df_temp, how='inner', left_index=True, right_index=True)

In [ ]:
merge

In [ ]:
# Calculate a numeric value for the age of the data, for use in color bar plot
def Dtime (row):
    #print(row.name)
    return (row.name-merge.index.max()).days
    
merge['DeltaDay']=merge.apply(lambda row: Dtime(row), axis=1)

In [ ]:
merge

In [ ]:
# Simpel fingure
ax=merge.plot(x='Middel', y='PosPct', title='Franz Plot (1. November 2020 - 15. Januar 2021)', style='.',label='');
ax.set_ylabel("Posetiv procent");
ax.set_xlabel("Middel temperature i dk");


In [ ]:
#nice figure which also incodes the time of the datapoints

cmap = cm.get_cmap('cool') # Colour map 
fig, ax = plt.subplots(1)

im=ax.scatter(merge.Middel, merge.PosPct, c=merge.DeltaDay, s=50, cmap=cmap, edgecolor='None')
plt.title('Franz Plot (1. November 2020 - 15. Januar 2021)')

# This is VERY manuelly made, should be generalized to automatically find nice places to put tic and labels
cbar=plt.colorbar(im, ticks=[-75, -45, -14])
cbar.ax.set_yticklabels(['2020-11-01', '2020-12-01', '2021-01-01'])

ax.set_ylabel("Corona Positiv procent (SSI data)");
ax.set_xlabel("Middeltemperature i Danmark (DMI data)");

plt.show()


#save a pdf for printing
#plt.savefig('SpecialPlot_Franz_20210115.pdf')